In [1]:
!pip install -q tqdm
!pip install -q flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import json, os, math, gc
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence
import torch

BASE_PATH          = "/content/drive/MyDrive"
FOLDER_NAME        = "HotpotQA_snapshot"
CHUNKS_FILENAME    = "all_docs_chunks.json"  
OUTPUT_FILENAME    = "all_docs_chunks_entities.json" 

CHUNKS_PATH  = os.path.join(BASE_PATH, FOLDER_NAME, CHUNKS_FILENAME)
OUTPUT_PATH  = os.path.join(BASE_PATH, FOLDER_NAME, OUTPUT_FILENAME)

assert os.path.isfile(CHUNKS_PATH), f"فایل قطعه‌ها پیدا نشد: {CHUNKS_PATH}"

print(f"📥 Loading chunks from: {CHUNKS_PATH}")
with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    chunks = json.load(f)
print(f"✅ Loaded {len(chunks):,} chunks")

Mounted at /content/drive
📥 Loading chunks from: /content/drive/MyDrive/HotpotQA_snapshot/all_docs_chunks.json
✅ Loaded 66,237 chunks


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tagger = SequenceTagger.load("flair/ner-english-ontonotes-large").to(device)
print("🔥 Model ready on", device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-07-21 09:48:17,833 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY
🔥 Model ready on cuda


In [4]:
def flair_extract_entities(batch_texts):
    sentences = [Sentence(t) for t in batch_texts]
    tagger.predict(sentences, verbose=False, mini_batch_size=16)
    results = []
    for sent in sentences:
        ents = [{"text": span.text, "type": span.tag} for span in sent.get_spans('ner')]
        results.append(ents)
    return results

In [5]:
BATCH_SIZE = 64
all_entities = []
for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="🔎 NER extracting"):
    batch = [chunks[j]["doc_chunk"] for j in range(i, min(i+BATCH_SIZE, len(chunks)))]
    batch_ents = flair_extract_entities(batch)
    all_entities.extend(batch_ents)
    if (i // BATCH_SIZE) % 30 == 0:   # حدود هر 2هزار قطعه
        gc.collect()

assert len(all_entities) == len(chunks)
print("✅ Entity extraction finished")

🔎 NER extracting: 100%|██████████| 1035/1035 [1:48:40<00:00,  6.30s/it]

✅ Entity extraction finished


In [6]:
for rec, ents in zip(chunks, all_entities):
    rec["entities"] = ents    # ← لیست دیکشنری [{text,type}, …]



In [7]:
print(f"💾 Saving augmented chunks to: {OUTPUT_PATH}")
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False)
print("📏 File size (MB):", round(os.path.getsize(OUTPUT_PATH)/(1024*1024), 2))
print("🎉 Done!")

💾 Saving augmented chunks to: /content/drive/MyDrive/HotpotQA_snapshot/all_docs_chunks_entities.json
📏 File size (MB): 124.06
🎉 Done!


In [8]:
import random, pprint
for rec in random.sample(chunks, 3):
    print("="*100)
    print(f"id={rec['id']} | title={rec['title_chunk']}")
    pprint.pprint(rec["entities"][:10])   # فقط ۱۰ مورد اول از هر قطعه

id=41454 | title=Mikhail Gorbachev
[{'text': '9 February 1990', 'type': 'DATE'},
 {'text': 'James Baker', 'type': 'PERSON'},
 {'text': 'US', 'type': 'GPE'},
 {'text': 'state', 'type': 'ORG'},
 {'text': 'NATO', 'type': 'ORG'},
 {'text': 'Baker', 'type': 'PERSON'},
 {'text': 'Gorbachev', 'type': 'PERSON'},
 {'text': 'May 1990', 'type': 'DATE'},
 {'text': 'US', 'type': 'GPE'},
 {'text': 'Bush', 'type': 'PERSON'}]
id=53289 | title=Métis
[{'text': 'the United States', 'type': 'GPE'},
 {'text': 'the Turtle Mountain Indian Reservation', 'type': 'LOC'},
 {'text': 'North Dakota', 'type': 'GPE'},
 {'text': 'Michif', 'type': 'LANGUAGE'},
 {'text': 'the Métis', 'type': 'NORP'},
 {'text': 'Chippewa', 'type': 'LOC'},
 {'text': 'Ojibwe', 'type': 'GPE'},
 {'text': 'years', 'type': 'DATE'},
 {'text': 'Métis', 'type': 'NORP'},
 {'text': 'Me', 'type': 'LANGUAGE'}]
id=31443 | title=EastEnders
[{'text': '2018', 'type': 'DATE'},
 {'text': 'EastEnders', 'type': 'WORK_OF_ART'},
 {'text': '20 per cent', 'typ